In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5

In [2]:
TickDataname='GDF2022_3_27_16_52.csv'
TickGDF=pd.read_csv('CollectedData/'+TickDataname)
Dataname='6feat'

TGDF=TickGDF[(TickGDF['TM_GP']>=0)&(TickGDF['TM_GP']<39)]
TGDF=TGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()
new_df=new_df.reset_index()
Train_df=new_df
Train_df.loc[new_df['TM_GP']==0,'시가']=Train_df.loc[new_df['TM_GP']==1,'시가'].values

In [3]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [4]:
qtylist=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량']

Train_df[qtylist]=Train_df[qtylist].div(Train_df['10단계호가합계잔량'],axis=0).values

Train_df['10단계호가합계잔량']=np.log(new_df['10단계호가합계잔량'])

In [5]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['매도총호가잔량', '매수총호가잔량','고가','저가','직전체결가격','시가']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']


In [6]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [7]:
Train_df[loglist]=np.log(Train_df[loglist])

In [8]:
ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickGDF=pd.read_csv('CollectedData/'+ORDTickDataname)

In [9]:
ORDTickGDF.columns

Index(['Unnamed: 0', 'ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO', 'ASKBID_TP_CD',
       'MODCANCL_TP_CD', 'ORD_QTY', 'ORD_ACPT_TM', 'TM_GP'],
      dtype='object')

In [10]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]

OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])

In [11]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [12]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

In [13]:
feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [14]:
feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [15]:
mbrnlist1=[(5,194),(2,155),(12,100),(17,29),(42,1),(44,1),(50,92),(2,83),(4,10118),(8,298)]
mbrnlist2=[(5,194),(12,100),(2,155),(17,29),(42,1),(44,1),(2,83),(4,10118),(4,9997),(50,91)]
mbrnlist=mbrnlist1+[a for a in mbrnlist2 if a not in mbrnlist1]

In [16]:
Train_df[feat_cols[:-1]]

,매도총호가잔량,매수총호가잔량,고가,저가,직전체결가격
0,13.306997,12.575422,0.002325,-0.007177,-0.003820
1,13.545755,12.766560,0.002384,-0.016230,-0.011201
2,13.670287,12.868486,0.002384,-0.017360,-0.013822
3,13.695710,12.951151,0.002384,-0.020008,-0.015662
4,13.703879,13.025584,0.002384,-0.021714,-0.014833
...,...,...,...,...,...
81895,14.604028,12.387069,0.031286,-0.020619,0.015968
81896,14.608903,12.376941,0.033448,-0.020619,0.021370
81897,14.606008,12.517950,0.033448,-0.020619,0.015266
81898,14.638396,12.491945,0.033448,-0.020619,0.009809


In [17]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    

    SaveDirPath='/Data/Min/TransLOB/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+str(now.year)+'_'+str(now.month)+'_'+str(now.day)
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix
    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')


5 194


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-33328534.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_5_1946feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__5_1946feat
2 155
1011342.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_1556feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_1556feat
12 100
-27072832.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_12_1006feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__12_1006feat
17 29


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-27191045.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_17_296feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__17_296feat
42 1
2665621.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_42_16feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__42_16feat
44 1
-17303050.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_44_16feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__44_16feat
50 92


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

-2678352.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_926feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__50_926feat
2 83
-13119612.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_2_836feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__2_836feat
4 10118
-13138160.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_4_101186feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__4_101186feat
8 298
-3971686.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_8_2986feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__8_2986feat
4 9997
-13028711.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_4_99976feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__4_99976feat
50 91
-6831958.0
path: /Data/Min/TransLOB/TrainData/Train_ORD_50_916feat
path: /Data/Min/TransLOB/TrainData/Train_ORD_Label__50_916feat


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ ==

In [19]:
Train_df[feat_cols].values.shape

(81900, 6)

In [21]:
Train_df['ylabel'].values

array([2., 0., 2., ..., 0., 1., 1.])

a

SaveDirPath='/Data/Min/TransLOB/TrainData/'
now=datetime.now()
TrainDataname='Train_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
TrainLabelDataname='Train_Label_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
np.save(SaveDirPath+TrainLabelDataname,increment)
with open(SaveDirPath+'DataInformation.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname)
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')
print("path:",SaveDirPath+TrainDataname)
print("path:",SaveDirPath+TrainLabelDataname)